In [1]:
!pip install rich

In [9]:
import pickle
import json
import random
from collections import Counter
import json
from rich.progress import track, Progress
from rich import print
import re
import pandas as pd
import sqlite3
import pandas as pd
import copy
import langid
from nervaluate import Evaluator
import pandas as pd

# RELOAD Pickled Variables

In [2]:
import pickle
def reload_vars(filename):
    this_var = None
    with open(filename, 'rb') as file:
        this_var = pickle.load(file)
        
    return this_var

# PICKLE Variables

In [3]:
def pkl_vars(varname, filename):
    with open(filename, 'wb') as file:
        pickle.dump(varname, file)

# Text to spacy list

In [21]:
with open('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-20250414173316.json') as f:
    mistralv3_json = json.load(f)
print(len(mistralv3_json))

1000

In [44]:
print(gt_text[156:])

/ Fürstl. <GPE-DES>Braunschw. und Lünäburg</GPE-DES> . wolbestelten Zehnd : Gegenschreibers auff der 
<GPE-SUBJ>Fürstl. Bergstadt Claußthal</GPE-SUBJ> / angetretenen andern Eh / Mit der ... Jungfer 
<PERSON-SUBJ>Christinen</PERSON-SUBJ> / Des ... Herrn <PERSON-SUBJ>Matthias Tollens</PERSON-SUBJ> / Fürstl. 
<GPE-DES>Braunschw. Lünäburg</GPE-DES> . Richter und Hütten-Reuter daselbst / hertzgeliebten Tochter ; Bey 
feyerlicher Vollziehung den <DATE-SUBJ>2. May-Monats</DATE-SUBJ> im <YEAR-SUBJ>1652sten</YEAR-SUBJ> Jahre

In [6]:
import re
#p = re.compile("<PERSON-SUBJ.*?>(.+?)</PERSON-SUBJ>")

def text_to_ents(title, text, include_text=False):
    ents_list = []
    p = re.compile("<[A-Z\-^>^<]*?>(.+?)</[A-Z\-^>^<]*>")
    
    curr_pos = 0
    for m in p.finditer(text):
        #print(curr_pos, m.start(), m.end(), m.group())
    
        match_text = m.group()
        len_match = len(match_text)
    
        start_tag_end_pos = m.group().find('>')
    
        tag = match_text[1:start_tag_end_pos]    
        end_tag_start_pos = match_text.find('</')
    
        value_text = match_text[start_tag_end_pos+1:end_tag_start_pos]
    
        if curr_pos == 0:
            actual_start_pos = m.start()
        else:
            actual_start_pos = title[curr_pos:].find(value_text) + curr_pos
            
        actual_end_pos = actual_start_pos + len(value_text)
    
        curr_pos = actual_end_pos + 1

        ent = {'label':tag,
               'start': actual_start_pos,
               'end': actual_end_pos}

        if include_text:
            ent['text'] = value_text
            print(ent)
        ents_list.append(ent)
    return ents_list

In [102]:
gt_ents = text_to_ents(title, gt_text, include_text=True)
print(len(gt_ents))

{'label': 'PERSON-SUBJ', 'start': 103, 'end': 116, 'text': 'von Berlepsch'}

{'label': 'LITWORK', 'start': 147, 'end': 178, 'text': 'Göttingischen gelehrten Zeitung'}

{'label': 'DATE-SUBJ', 'start': 178, 'end': 203, 'text': '16. Febr. <YEAR-SUBJ>1797'}

3

In [103]:
pred_ents = text_to_ents(title, pred_text)
print(len(pred_ents))

2

In [104]:
gt_labels = []
pred_labels = []

for elem in mistralv3_json:
    sent_id = elem['sent_id']
    title = elem['title']
    gt_text = elem['gt']
    pred_text = elem['pred']

    gt_labels.append(text_to_ents(title, gt_text))
    pred_labels.append(text_to_ents(title, pred_text))

# Nervaluate - Mistral

In [93]:
!pip install nervaluate

In [11]:
tags = ['DATE-CREATION',
 'DATE-PUB',
 'DATE-SUBJ',
 'GPE-AUT',
 'GPE-CREATION',
 'GPE-DES',
 'GPE-PUB',
 'GPE-SUBJ',
 'LITWORK',
 'ORG-AUT',
 'ORG-CREATION',
 'ORG-DES',
 'ORG-PUB',
 'ORG-SUBJ',
 'PERSON-AUT',
 'PERSON-CON',
 'PERSON-EDT',
 'PERSON-PUB',
 'PERSON-SUBJ',
 'YEAR-CREATION',
 'YEAR-PUB',
 'YEAR-SUBJ']

In [5]:
from collections import defaultdict

def flip_nested_dict(dd):
    result = defaultdict(dict)
    for k1, d in dd.items():
        for k2, v in d.items():
            result[k2][k1] = v
    return dict(result)

In [8]:
from nervaluate import Evaluator
import pandas as pd

evaluator = Evaluator(gt_labels, pred_labels, tags=tags, loader="default")

NameError: name 'gt_labels' is not defined

In [108]:
results = evaluator.evaluate()

macro_results = results[0]
micro_results = results[1]

pd_macro = pd.DataFrame(flip_nested_dict(macro_results))
pd_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,1735,1017,0,1894,1438,4646,4190,0.414081,0.373440,0.392712
partial,1579,0,1173,1894,1438,4646,4190,0.516826,0.466100,0.490154
strict,1061,1691,0,1894,1438,4646,4190,0.253222,0.228368,0.240154
exact,1579,1173,0,1894,1438,4646,4190,0.376850,0.339862,0.357402


In [109]:
pd_micro = pd.DataFrame(flip_nested_dict(micro_results))
pd_micro

,ent_type,partial,strict,exact
DATE-CREATION,"{'correct': 19, 'incorrect': 51, 'partial': 0,...","{'correct': 14, 'incorrect': 0, 'partial': 56,...","{'correct': 3, 'incorrect': 67, 'partial': 0, ...","{'correct': 14, 'incorrect': 56, 'partial': 0,..."
DATE-PUB,"{'correct': 6, 'incorrect': 55, 'partial': 0, ...","{'correct': 20, 'incorrect': 0, 'partial': 41,...","{'correct': 2, 'incorrect': 59, 'partial': 0, ...","{'correct': 20, 'incorrect': 41, 'partial': 0,..."
DATE-SUBJ,"{'correct': 160, 'incorrect': 76, 'partial': 0...","{'correct': 113, 'incorrect': 0, 'partial': 12...","{'correct': 91, 'incorrect': 145, 'partial': 0...","{'correct': 113, 'incorrect': 123, 'partial': ..."
GPE-AUT,"{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '..."
GPE-CREATION,"{'correct': 30, 'incorrect': 55, 'partial': 0,...","{'correct': 78, 'incorrect': 0, 'partial': 7, ...","{'correct': 27, 'incorrect': 58, 'partial': 0,...","{'correct': 78, 'incorrect': 7, 'partial': 0, ..."
GPE-DES,"{'correct': 450, 'incorrect': 159, 'partial': ...","{'correct': 434, 'incorrect': 0, 'partial': 17...","{'correct': 346, 'incorrect': 263, 'partial': ...","{'correct': 434, 'incorrect': 175, 'partial': ..."
GPE-PUB,"{'correct': 10, 'incorrect': 27, 'partial': 0,...","{'correct': 22, 'incorrect': 0, 'partial': 15,...","{'correct': 7, 'incorrect': 30, 'partial': 0, ...","{'correct': 22, 'incorrect': 15, 'partial': 0,..."
GPE-SUBJ,"{'correct': 127, 'incorrect': 96, 'partial': 0...","{'correct': 160, 'incorrect': 0, 'partial': 63...","{'correct': 90, 'incorrect': 133, 'partial': 0...","{'correct': 160, 'incorrect': 63, 'partial': 0..."
LITWORK,"{'correct': 19, 'incorrect': 39, 'partial': 0,...","{'correct': 14, 'incorrect': 0, 'partial': 44,...","{'correct': 9, 'incorrect': 49, 'partial': 0, ...","{'correct': 14, 'incorrect': 44, 'partial': 0,..."
ORG-AUT,"{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '..."


## Llama 2-shot

In [115]:
with open('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-2shot-20250414181340.json') as f:
    llama_json = json.load(f)
print(len(llama_json))

1000

In [111]:
llama_gt_labels = []
llama_pred_labels = []

for elem in llama_json:
    sent_id = elem['sent_id']
    title = elem['title']
    gt_text = elem['gt']
    pred_text = elem['pred']

    llama_gt_labels.append(text_to_ents(title, gt_text))
    llama_pred_labels.append(text_to_ents(title, pred_text))

In [113]:
evaluator = Evaluator(llama_gt_labels, llama_pred_labels, tags=tags, loader="default")
llama_results = evaluator.evaluate()
llama_macro_results = llama_results[0]
llama_micro_results = llama_results[1]

llama_pd_macro = pd.DataFrame(flip_nested_dict(llama_macro_results))
llama_pd_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,1726,1577,0,1342,2264,4645,5567,0.310041,0.371582,0.338034
partial,1812,0,1491,1342,2264,4645,5567,0.459404,0.550592,0.500881
strict,1038,2265,0,1342,2264,4645,5567,0.186456,0.223466,0.203290
exact,1812,1491,0,1342,2264,4645,5567,0.325489,0.390097,0.354877


In [114]:
llama_pd_micro = pd.DataFrame(flip_nested_dict(llama_micro_results))
llama_pd_micro

,ent_type,partial,strict,exact
DATE-CREATION,"{'correct': 60, 'incorrect': 20, 'partial': 0,...","{'correct': 18, 'incorrect': 0, 'partial': 62,...","{'correct': 12, 'incorrect': 68, 'partial': 0,...","{'correct': 18, 'incorrect': 62, 'partial': 0,..."
DATE-PUB,"{'correct': 2, 'incorrect': 62, 'partial': 0, ...","{'correct': 21, 'incorrect': 0, 'partial': 43,...","{'correct': 1, 'incorrect': 63, 'partial': 0, ...","{'correct': 21, 'incorrect': 43, 'partial': 0,..."
DATE-SUBJ,"{'correct': 156, 'incorrect': 83, 'partial': 0...","{'correct': 109, 'incorrect': 0, 'partial': 13...","{'correct': 86, 'incorrect': 153, 'partial': 0...","{'correct': 109, 'incorrect': 130, 'partial': ..."
GPE-AUT,"{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '..."
GPE-CREATION,"{'correct': 76, 'incorrect': 24, 'partial': 0,...","{'correct': 86, 'incorrect': 0, 'partial': 14,...","{'correct': 69, 'incorrect': 31, 'partial': 0,...","{'correct': 86, 'incorrect': 14, 'partial': 0,..."
GPE-DES,"{'correct': 454, 'incorrect': 311, 'partial': ...","{'correct': 538, 'incorrect': 0, 'partial': 22...","{'correct': 349, 'incorrect': 416, 'partial': ...","{'correct': 538, 'incorrect': 227, 'partial': ..."
GPE-PUB,"{'correct': 2, 'incorrect': 39, 'partial': 0, ...","{'correct': 26, 'incorrect': 0, 'partial': 15,...","{'correct': 2, 'incorrect': 39, 'partial': 0, ...","{'correct': 26, 'incorrect': 15, 'partial': 0,..."
GPE-SUBJ,"{'correct': 40, 'incorrect': 254, 'partial': 0...","{'correct': 213, 'incorrect': 0, 'partial': 81...","{'correct': 26, 'incorrect': 268, 'partial': 0...","{'correct': 213, 'incorrect': 81, 'partial': 0..."
LITWORK,"{'correct': 55, 'incorrect': 44, 'partial': 0,...","{'correct': 26, 'incorrect': 0, 'partial': 73,...","{'correct': 20, 'incorrect': 79, 'partial': 0,...","{'correct': 26, 'incorrect': 73, 'partial': 0,..."
ORG-AUT,"{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '...","{'correct': 0, 'incorrect': 0, 'partial': 0, '..."


# 0-Shot

In [2]:
def eval_it(results_file):

    with open(results_file) as f:
        results_json = json.load(f)

    gt_labels = []
    pred_labels = []
    
    for elem in results_json:
        sent_id = elem['sent_id']
        title = elem['title']
        gt_text = elem['gt']
        pred_text = elem['pred']
    
        gt_labels.append(text_to_ents(title, gt_text))
        pred_labels.append(text_to_ents(title, pred_text))

    evaluator = Evaluator(gt_labels, pred_labels, tags=tags, loader="default")
    eval_results = evaluator.evaluate()
    macro_results = eval_results[0]
    micro_results = eval_results[1]
    
    pd_macro = pd.DataFrame(flip_nested_dict(macro_results))
    pd_micro = pd.DataFrame(flip_nested_dict(micro_results))

    return pd_macro, pd_micro

In [118]:
mistralv3_0shot_macro, mistralv3_0shot_micro = eval_it('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-0shot-20250416065545.json')
mistralv3_0shot_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,341,3073,0,1236,3323,4650,6737,0.050616,0.073333,0.059893
partial,159,0,3255,1236,3323,4650,6737,0.265177,0.384194,0.313779
strict,61,3353,0,1236,3323,4650,6737,0.009054,0.013118,0.010714
exact,159,3255,0,1236,3323,4650,6737,0.023601,0.034194,0.027927


In [119]:
llama3_0shot_macro, llama3_0shot_micro = eval_it('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-0shot-20250416075022.json')
llama3_0shot_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,381,1041,0,3223,1210,4645,2632,0.144757,0.082024,0.104713
partial,495,0,927,3223,1210,4645,2632,0.364172,0.206351,0.263433
strict,173,1249,0,3223,1210,4645,2632,0.065729,0.037244,0.047547
exact,495,927,0,3223,1210,4645,2632,0.188070,0.106566,0.136045


# 3-shot

In [12]:
mistralv3_3shot_macro, mistralv3_3shot_micro = eval_it('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-3shot-20250417193508.json')
mistralv3_3shot_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,1559,1697,0,1526,1618,4782,4874,0.319860,0.326014,0.322908
partial,1555,0,1701,1526,1618,4782,4874,0.493537,0.503032,0.498239
strict,1018,2238,0,1526,1618,4782,4874,0.208863,0.212882,0.210853
exact,1555,1701,0,1526,1618,4782,4874,0.319040,0.325178,0.322080


In [13]:
llama3_3shot_macro, llama3_3shot_micro = eval_it('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-3shot-20250417193606.json')
llama3_3shot_macro

,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1
ent_type,1676,1518,0,1589,3113,4783,6307,0.265736,0.350408,0.302254
partial,1748,0,1446,1589,3113,4783,6307,0.391787,0.516621,0.445627
strict,1090,2104,0,1589,3113,4783,6307,0.172824,0.227890,0.196573
exact,1748,1446,0,1589,3113,4783,6307,0.277152,0.365461,0.315239


# Re-evaluate

In [2]:
# https://www.newscatcherapi.com/blog/ultimate-guide-to-text-similarity-with-python
def jaccard_similarity(x,y):
    """ returns the jaccard similarity between two lists """
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [18]:
import re

def remove_tags(annotation):
    clean = re.sub(r"<\/?[A-Z\-^>^<]*?>", "", annotation)
    return clean

In [22]:
def compute_jaccard(results_file):

    with open(results_file) as f:
        results_json = json.load(f)

    jd = 0
    
    for elem in results_json:
        sent_id = elem['sent_id']
        title = elem['title']
        gt_text = elem['gt']
        pred_text = elem['pred']
        
        orig_gt_text = remove_tags(gt_text)
        orig_pred_text = remove_tags(pred_text)

        jd += jaccard_similarity(orig_gt_text, orig_pred_text)

    jd = jd/len(results_json)

    print(jd)
    return (jd)

In [20]:
compute_jaccard('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-2shot-20250414181340.json')

0.9708069648797011


In [23]:
compute_jaccard('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-2shot-20250414173316.json')

0.9679289331390154


0.9679289331390154

# Sentence Similarity

In [27]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [36]:
model = SentenceTransformer("all-mpnet-base-v2")

def compute_cossim(results_file):

    with open(results_file) as f:
        results_json = json.load(f)

    cossim = 0
    
    for elem in results_json:
        sent_id = elem['sent_id']
        title = elem['title']
        gt_text = elem['gt']
        pred_text = elem['pred']
        
        orig_gt_text = remove_tags(gt_text)
        orig_pred_text = remove_tags(pred_text)

        sentences = [orig_gt_text, orig_pred_text]
        
        embeddings = model.encode(sentences)
        similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
        
        cossim += similarity.item()

    cossim = cossim/len(results_json)

    print(cossim)
    return (cossim)

In [38]:
compute_cossim('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-0shot-20250416065545.json')
compute_cossim('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-2shot-20250414173316.json')
compute_cossim('./data/ner-202407/results-Mistral-7B-Instruct-v0.3-3shot-20250417193508.json')

0.7932192219495773
0.9531658143997193
0.9662463219165802


0.9662463219165802

In [37]:
compute_cossim('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-0shot-20250416075022.json')
compute_cossim('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-2shot-20250414181340.json')
compute_cossim('./data/ner-202407/results-Meta-Llama-3-8B-Instruct-3shot-20250417193606.json')

0.9169458716511726
0.970206836372614
0.9009330737590789


0.9009330737590789